# DS102 | In Class Practice Week 3C - Data Visualisation II
<hr>

## Learning Objectives
At the end of the lesson, you will be able to:

- create histograms using `matplotlib` library and `seaborn` library

- use `pd.cut()` to store values in each bin and use the result to interpret a histogram plot

- create a stacked bar chart using `matplotlib` library and `seaborn` library

- create multiple plots in the same diagram using `add_subplot()`

- create a wordcloud using the `wordcloud` library

- plot a `jointplot` to show the joint distributions of 2 metrics


## Datasets Required for this In Class

In the order they are introduced: 
1. `loans-25k.csv`

2. `loans-descs-100.csv`

In [ ]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Display all floats rounded off to 1 decimal place
pd.options.display.float_format = '{:,.2f}'.format
# Plot inline in Jupyter notebook
%matplotlib inline
# Settings throughout the notebook
sns.set()
# Width = 16, Height = 6
DIMS=(16, 6)

import warnings
warnings.filterwarnings('ignore')

Read from `loans-25k.csv` into `loans_df`. Find the properties of the `loans_df`.

In [ ]:
## Dataset 1: Credits at the end of the notebook
loans_df = pd.read_csv('loans-25k.csv')
display(loans_df.describe())
display(loans_df.info())

## Histogram

A histogram is used to plot the **count** of individual observations within a bin. Each bin has **a specified range** as determined by the position and width of the bar.

Note: If you see the warning `The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.`, you can ignore it.

https://seaborn.pydata.org/generated/seaborn.distplot.html

In [ ]:
fig = plt.figure(figsize=(18, 8))
ax1 = fig.add_subplot(111)

# Plot a histogram for 'loan_amnt'. Use 15 bins.
# Use sns.distplot here. Also, set kde=False as we do not need the Kernel Density Estimate plot shape.
# Remember to pass in the axes object.

# Set the title and y-axis label.
ax1.set_title("Histogram of loan amount (USD), 15 bins", fontsize=16)
ax1.set_ylabel("No. of borrowers")

plt.show()

To understand the left and right limits of each bin, use `pd.cut(Series, bins=bins)`. Store the output in a new column `loan_amnt_bin`. 

In [ ]:
# Fill in parameters for pd.cut(), applying it on 'loan_amnt' column. Use 15 bins.
loans_df_bin = loans_df.copy()
loans_df_bin['loan_amnt_bin'] = pd.cut(...)
loans_df_bin.head()

Counting the number of observations in each bin, gives the height of each bar in the histogram. The limits of each bin determine the position and width of the bar in the histogram. Use `df.groupby()` and `size()` to get the number of observations in each bin. 

In [ ]:
df_bins_count = loans_df_bin.groupby('loan_amnt_bin').size().reset_index(name='n')
df_bins_count

## Stacked Bar Chart

A stacked bar chart can be used to see the number of observations within a cohort, and the cohort's cumulative sum. In this case, we want to see how many loans belonging to each grade divided by its different loan terms. Hence, we group the data by these 2 columns. Use `df.groupby()` and `size()` to perform this counting. Note that `df.groupby()` can take also take in a `list` as its parameter.

In [ ]:
term_grade_loans_df = loans_df.copy()
term_grade_loans_df = term_grade_loans_df[['id', 'term', 'grade']]

# This line uses groupby() to count the number of loans belonging to each term and grade
term_grade_loans_agg = term_grade_loans_df.groupby(['term', 'grade']).size().reset_index(name="No. of Loans")
term_grade_loans_agg

With the aggregated `term_grade_loans_df`, now, represent each `term` as one column. Then, have one column for `36 months` and one column for `60 months` respectively. Additionally, add a column for `grade`. Use `pd.pivot_table()` for this step.

In [ ]:
# Complete the pivot_table() method so that rows and columns and 'grade' and 'term' respectively. 
# Here, we are interested in the total number of loans for each grade and term
term_grade_loans_pivot = pd.pivot_table(term_grade_loans_agg, index=[''], columns=[''],
                                        values='', aggfunc=)

# This chunk resets the index and set the column names to: 'grade', '36 months', '60 months'
term_grade_loans_pivot = term_grade_loans_pivot.reset_index()
term_grade_loans_pivot.columns = ['grade', '36_months', '60_months']
term_grade_loans_pivot

### A Small Detour: Multiple subplots

Before plotting the stacked bar graph, plot the individual bar graphs. To have the same plots side-by-side, use `fig.add_subplot()`.

There are 3 parameters for this function, all as `int`s. The first is the number of rows, the second is number of columns and the last is the position of the plot. From the top-left, the plots are numbered from 1 and increase as you move right, when the rightmost plot is reached, go to the next row and start from the left again. 

In this example, there is 1 row and 2 columns. The first plot (position `1`) is the plot on the left and the second plot (position `2`) is the plot on the right. Each of them will be assigned to an `AxePlot` with variables `ax1` and `ax2`.

Note that when you create individual plots, set the `ax` parameter to the `ax1` or `ax2` variable. 

Also, when plots are side-by-side, **the $y$-axis should reflect the same scale**.

https://seaborn.pydata.org/generated/seaborn.barplot.html

In [ ]:
fig = plt.figure(figsize=(16, 7))

# Recap: explanation for fig.add_subplot()
# The first number is the no. of rows. 
# The second number is the no. of columns.
# The third number is the plot number. Count right, then count down.
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

# Complete sns.barplot() to plot for 36 months with color 'tomato'
sns.barplot(x=, y=, ax=, color='tomato')
ax1.set_ylabel("No. of Loans")
ax1.set_title("No. of Loans by Grade, 36 Months Term")

# Complete sns.barplot() to plot for 60 months with color 'skyblue'
sns.barplot(x=, y=, ax=, data=, color='skyblue')
ax2.set_title("No. of Loans by Grade, 60 Months Term")
ax2.set_ylabel("No. of Loans")

# Set the scale of the plots to be the same.
ax1.set_yticks(np.arange(0, 7500, 1000))
ax2.set_yticks(np.arange(0, 7500, 1000))

# Show the graph
plt.show()

To plot the stacked bar graph, simply use `stacked=True` when you plot the graph from the `DataFrame`. Note that for this, the column that you would like to be the $x$-axis **needs** to be the index. Set the index by using `set_index`.

In [ ]:
term_grade_loans_pivot_grade_i = term_grade_loans_pivot.set_index('grade')
term_grade_loans_pivot_grade_i

In [ ]:
# Run this chunk to plot the stacked bar chart here. Note that stacked=True
fig, ax1 = plt.subplots()
term_grade_loans_pivot_grade_i.plot(kind='bar', stacked=True, figsize=DIMS, 
                                    title="No. of Loans by Grade", ax=ax1)
ax1.set_ylabel("No. of Loans")
plt.show()

## Wordcloud

A wordcloud is useful to find out what are common words in a corpus of texts. A corpus is simply a collection of text documents. Before building a wordcloud, first collect all the texts.

https://amueller.github.io/word_cloud/auto_examples/colored.html

In [ ]:
# Dataset 2: Credits at the end of the notebook
desc_df = pd.read_csv('loans-descs-100.csv', sep='|')
desc_df.sample(3) # Sample 3 of the records

In [ ]:
descriptions_series = desc_df['desc']

In [ ]:
# Repeatedly execute this cell to look at the text of each desc.
descriptions_series.sample(1).iloc[0]

To build a wordcloud, all the descriptions need to be connected to form a long string. But, some cleaning needs to be done first. The next steps are used to collect these words:

1. Remove `Borrower added on.. >` in the beginning of the string
2. Remove HTML tags from the descriptions
3. Remove special characters from the descriptions. Remove trailing and leading spaces, convert all words to lower case

In [ ]:
descriptions_list = descriptions_series.tolist() # convert series to list
descriptions_list[0:5]

In [ ]:
clean_descriptions = []
for desc in descriptions_list:
    # Remove the first bunch of letters
    desc = re.sub('Borrower added on (\d/?)+\s>', '', desc)
    
    # Remove HTML tags
    desc = re.sub('<[a-z]+/?>', '', desc)
    
    # Remove special characters
    desc = re.sub('[-_,$&!.;%]', '', desc)
    
    # Convert to lower case, remove leading and trailing spaces
    desc = desc.strip().lower()
    
    clean_descriptions.append(desc)

clean_descriptions[0:5]

Finally when all the words are cleaned, concatenate them to form a long string. Finally, use `WordCloud` to plot the wordcloud. Note that the larger the word, the more commmon the word appears in the corpus.

In [ ]:
long_str_descriptions = " ".join(clean_descriptions)
long_str_descriptions[:500]

Now, we are ready to use our preprocessed text data to create a wordcloud!

https://amueller.github.io/word_cloud/generated/wordcloud.WordCloud.html#wordcloud.WordCloud

https://matplotlib.org/api/_as_gen/matplotlib.pyplot.imshow.html

In [ ]:
plt.figure(figsize=(15,8))

# create wordcloud here
plt.figure(figsize=(15,8))
wc = WordCloud(width=400, height=150, background_color="white", max_words=150, relative_scaling=)
desc_wordcloud = wc.generate(...)

plt.imshow(...)
plt.axis("off")
plt.title("Wordcloud of loan descriptions", fontsize=20)
plt.show()

## Self-Study: Jointplot

A jointplot is used to see the **joint observation** of datapoints. One axis will show the distribution of one metric while the other axis will show the distribution of the other metric. 

Each hexagon **exists in a 2 dimensional space** that represents a bin corresponding to the **<u>concurrent</u> observation of $X=x$ and $Y=y$**. In this plot, the darker the hexagon, the more observations are in that bin.

https://seaborn.pydata.org/generated/seaborn.jointplot.html

In [ ]:
## Dataset 1: Credits at the end of the notebook
loans_df = pd.read_csv('loans-25k.csv')
loans_df.head()

In [ ]:
fig = plt.figure()
# Plot the joint plot here. Set the x, y and kind='hex' parameter
g = sns.jointplot(data=loans_df, x = 'loan_amnt', y = 'int_rate', kind='hex')
# Set the size of the plot and the title.
g.fig.suptitle('Loan Amount vs. Interest Rate of loans')
g.fig.set_size_inches(8,8)
# Shrink the plot by 10% so the title is visible.
plt.subplots_adjust(top=0.9)

# Additional Exercises
## Exercise 1

Given `loans_df` which reads from the same dataset, plot a histogram where the $x$-axis represents the different types of `home_ownership` and the $y$-axis represents the number of loans whose members have that particular home ownership. Filter only for `home_ownership` with the values `'MORTGAGE', 'OWN', 'RENT'`.

In [ ]:
# Exercise 1
# Write your code here
l_df = pd.read_csv('loans-25k.csv')
l_df = l_df[l_df['home_ownership'].isin(['MORTGAGE', 'OWN', 'RENT'])]
l_df_agg = l_df.groupby('home_ownership').size().reset_index(name='No. of Loans')
ax = l_df_agg.plot(kind='bar', x='home_ownership', y='No. of Loans', 
              title="No. of Loans by Home Ownership", figsize=DIMS)
plt.ylabel("No. of Loans")
plt.xlabel("Type of Home Ownership")
plt.show()

## Exercise 2

Given `term_grade_loans_pivot` which has the number of loans in each grade, per term, plot a clustered bar graph where each cluster on the $x$-axis represents a grade and the $y$-axis represents the number of loans.

In [ ]:
# l_df_agg_pivot is given to you.
l_df = pd.read_csv('loans-25k.csv')
l_df_agg = l_df.groupby(['term', 'grade']).size().reset_index(name="No. of Loans")
# Create the term_grade_loans_pivot where the index is grade and the columns are the different terms
l_df_agg_pivot = pd.pivot_table(l_df_agg, 
                                index=['grade'], columns=['term'],
                                values='No. of Loans', aggfunc=np.sum)
# Then reset the index and set the columns to be 3 columns: 'grade', '36 months', '60 months'
l_df_agg_pivot = l_df_agg_pivot.reset_index()
l_df_agg_pivot.columns = ['grade', '36 months', '60 months']
print(l_df_agg_pivot)

In [ ]:
# Exercise 2 - Plot the clustered bar plot. 
# Hint: Refer to Data Visualisation I to get the code for the clustered bar plot.
width=0.20

#Create the figure
fig = plt.figure(figsize=DIMS)

#Add the subplot
ax = fig.add_subplot(111)

#Plot the values
term_grade_loans_pivot.plot(kind='bar', x='grade', y='36 months', 
                            ax=ax, position=1, width=width, color='steelblue')
term_grade_loans_pivot.plot(kind='bar', x='grade', y='60 months', 
                            ax=ax, position=2, width=width, color='tomato')

# Your turn: Add the title of the plot
ax.set_title("No. of loans by grade, by term")

#Finally, show the plot
plt.show()

**Credits**
- [Kaggle](https://www.kaggle.com/wendykan/lending-club-loan-data) for Dataset 1 & 2
- [Kaggle](https://www.kaggle.com/monkeasy/exploring-gender-equality-in-us-wages/data) for Dataset 3